In [4]:
import requests
import pandas as pd
import time

client_id = "5643deda5e6f426ba4d354d6f8f6fed3"
client_secret = "*******************"

def get_token(client_id, client_secret):
    auth_url = "https://accounts.spotify.com/api/token"
    auth_response = requests.post(
        auth_url,
        data={"grant_type": "client_credentials"},
        auth=(client_id, client_secret)
    )
    return auth_response.json()["access_token"]

token = get_token(client_id, client_secret)

def search_tracks(query, token, limit=50):
    headers = {"Authorization": f"Bearer {token}"}
    url = "https://api.spotify.com/v1/search"
    params = {
        "q": query,
        "type": "track",
        "limit": limit
    }
    response = requests.get(url, headers=headers, params=params)
    if "tracks" not in response.json():
        print(f"Error for query: {query}")
        return []
    return response.json()["tracks"]["items"]

# List of artist search queries
artist_queries = ["coldplay", "beyonce", "eminem", "dua lipa", "radiohead", "arctic monkeys", "lady gaga", "drake", "bruno mars", "adele"]

all_tracks = []
for artist in artist_queries:
    items = search_tracks(artist, token, limit=50)
    for item in items:
        all_tracks.append({
            'track_name': item['name'],
            'artist': item['artists'][0]['name'],
            'album': item['album']['name'],
            'release_date': item['album']['release_date'],
            'popularity': item['popularity']
        })
    time.sleep(0.3)  # short delay to avoid rate limit

df = pd.DataFrame(all_tracks)
df.drop_duplicates(subset=["track_name", "artist"], inplace=True)
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
df = df.dropna(subset=['release_date'])
df = df[df['release_date'] >= '2010-01-01']

print(f"Final dataset size: {df.shape[0]} rows")
df.head()

Final dataset size: 308 rows


,track_name,artist,album,release_date,popularity
2,feelslikeimfallinginlove - Single Version,Coldplay,feelslikeimfallinginlove,2024-06-21,76
3,feelslikeimfallinginlove,Coldplay,Moon Music (Full Moon Edition),2024-10-03,74
4,A Sky Full of Stars,Coldplay,Ghost Stories,2014-05-16,90
6,ALL MY LOVE,Coldplay,Moon Music,2024-10-04,73
8,Something Just Like This,The Chainsmokers,Memories...Do Not Open,2017-04-07,90


In [5]:
# Save to CSV
df.to_csv("spotify_tracks_expanded.csv", index=False)

# Download the file
from google.colab import files
files.download("spotify_tracks_expanded.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>